In [63]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor

In [64]:
def rmserror_log(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [65]:
train = pd.read_csv('houses/clean_train.csv')
test = pd.read_csv('houses/clean_test.csv')
test_ID = test['Id']

del train['Id']
del test['Id']
train["SalePrice"] = np.log1p(train["SalePrice"])

In [ ]:
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, 
                     n_jobs=1, scoring=rmserror_log)
tpot.fit(train.drop('SalePrice', axis=1).values, train['SalePrice'])

Optimization Progress:  34%|███▎      | 101/300 [10:55<10:39,  3.21s/pipeline] 

Generation 1 - Current best internal CV score: 0.38439377171626965


Optimization Progress:  51%|█████     | 153/300 [24:25<02:07,  1.16pipeline/s]  

Generation 2 - Current best internal CV score: 0.38439377171626965


Optimization Progress:  68%|██████▊   | 203/300 [27:37<04:17,  2.66s/pipeline]

Generation 3 - Current best internal CV score: 0.38439377171626965


Optimization Progress:  71%|███████   | 213/300 [28:10<05:42,  3.94s/pipeline]

In [32]:
predictions = np.expm1(tpot.predict(test))
result_df = pd.DataFrame({'Id': test_ID, 'SalePrice': predictions})
result_df.to_csv('houses/predictions.csv', index=False)